In [ ]:
import pandas as pd
import cv2
from transformers import CLIPProcessor, CLIPModel
import os
import ast
from PIL import Image

In [4]:
def extract_images(fichier, image):
    df=pd.read_csv(fichier)
    liste_coords =df['box'].to_list()
    nom_image = image
    img = nom_image+".jpg"
    path=os.getcwd()
    img = cv2.imread(image)
    image = cv2.imread(path + "\\"+ img)
    
    if image is None:
            print(f"Erreur : impossible de charger l'image {img}")
            exit()
    if not os.path.exists(path+"\\"+nom_image):
        os.makedirs(nom_image)
    compteur=0
    for i in liste_coords:
        compteur+=1
        dico = ast.literal_eval(i)
        valeurs=list(dico.values())
        print(valeurs)
        y1, x1, y2, x2 = int(round(valeurs[0])), int(round(valeurs[1])), int(round(valeurs[2])), int(round(valeurs[3]))
        print(x1, x2, y1, y2)
        cropped_img= image[x1:x2,y1:y2 ]
        cv2.imwrite(f"{nom_image}\\{nom_image}_illus{compteur}.png", cropped_img)
            


In [ ]:
df= pd.read_csv('C:\\Users\\fanti\\Documents\\stage_highvision\\retrainyolomaison\\tests\\testphase2\\demonstration\\Excelsior19240710image1.jpg_annots.csv')

liste_coords =df['box'].to_list()
print(liste_coords)
path=os.getcwd()
nom_image ="Excelsior19240710image1"
img = nom_image+".jpg"
print(path + "\\"+ img)
image = cv2.imread(path + "\\"+ img)
if image is None:
    print(f"Erreur : impossible de charger l'image {nom_image}")
    exit()
compteur=0
if not os.path.exists(path+"\\"+nom_image):
    os.makedirs(nom_image)
for i in liste_coords:
    compteur+=1
    dico = ast.literal_eval(i)
    valeurs=list(dico.values())
    print(valeurs)
    y1, x1, y2, x2 = int(round(valeurs[0])), int(round(valeurs[1])), int(round(valeurs[2])), int(round(valeurs[3]))
    print(x1, x2, y1, y2)
    cropped_img= image[x1:x2,y1:y2 ]
    cv2.imwrite(f"{nom_image}\\{nom_image}_illus{compteur}.png", cropped_img)

In [ ]:
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

https://medium.com/@kerry.halupka/getting-started-with-openais-clip-a3b8f5277867

In [ ]:

def image_grid(imgs, cols):
    rows = (len(imgs) + cols - 1) // cols
    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid
path=os.getcwd()
nom_image="Excelsior19240710image1"
image_urls=[]
for i in range(1,20):
    image_urls.append(str(path+"\\"+nom_image+"\\"+nom_image+"_illus"+str(i)+".png"))
images = []
try:
    for url in image_urls:
        images.append(Image.open(url))
except:
    pass

grid = image_grid(images, cols=3)
display(grid)

In [ ]:
fichier_texte=str('texte'+nom_image+".txt")
with open (fichier_texte,'r', encoding="utf-8") as ft:
   texte=ft.read()
   liste_zones=texte.split("ZONE_TEXTE")
   
   liste2=[]
   for i in range(0, len(liste_zones)):
      
      if len(liste_zones[i])>= 10 and len(liste_zones[i])<=77:
         terme=liste_zones[i].replace("\n","")
         terme=liste_zones[i].replace("- ","")
         liste2.append(terme)
print(liste2)       
 

In [9]:
inputs = processor(text=liste2, images=images, return_tensors="pt", padding=True, do_convert_rgb=False)

outputs = model(**inputs)
logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(8, 20))

for idx in range(len(images)):

    # show original image
    fig.add_subplot(len(images), 2, 2*(idx+1)-1 )
    plt.imshow(images[idx])
    plt.xticks([])
    plt.yticks([])

    # show probabilities
    fig.add_subplot(len(images), 2, 2*(idx+1))
    
    plt.barh(range(len(probs[0].detach().numpy())),probs[idx].detach().numpy(), tick_label=liste2)
    plt.xlim(0,1.0)

    plt.subplots_adjust(left=0.1,
                        bottom=0.1,
                        right=0.9,
                        top=0.9,
                        wspace=0.2,
                        hspace=0.8)

plt.show()

In [13]:
len(probs)
liste_probas =probs.tolist()

In [ ]:
print (liste_probas)
for i in liste_probas:
    print(i)

In [15]:
probabs_image1=[]
for probab in liste_probas[0]:
    if probab>=0.4:
        probabs_image1.append(liste_probas[0].index(probab))
print(probabs_image1)

[9]


In [ ]:
liste2[9]

In [17]:
liste_probas_par_ordre_images=[]

for i in range(0,len(liste_probas)):
    indices_probas=[]
    for probab in liste_probas[i]:
        if probab>=0.2:
            indices_probas.append(liste_probas[i].index(probab))
    liste_probas_par_ordre_images.append(indices_probas)


Dans la cellule ci dessous, l'indice correspond au numéro de l'image, les chiffres à l'indice dans la liste des morceaux de texte des textes associés à plus de 0,2.

In [ ]:
print(liste_probas_par_ordre_images)

In [ ]:
compteur=1
for i in liste_probas_par_ordre_images:
    print(f"image n{compteur}")
    compteur+=1
    for y in i:
        print(liste2[y])